In [147]:
import pandas as pd
import numpy as np
import math 
import string
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [148]:
url = 'https://raw.githubusercontent.com/PersieB/AshTutor/master/reviews.csv'
dataset = pd.read_csv(url)

In [149]:
dataset.head(20)

,Id,Review,Label
0,0,good and interesting,5
1,1,"This class is very helpful to me. Currently, I...",5
2,2,like!Prof and TAs are helpful and the discussi...,5
3,3,Easy to follow and includes a lot basic and im...,5
4,4,Really nice teacher!I could got the point eazl...,4
5,5,"Great course - I recommend it for all, especia...",5
6,6,One of the most useful course on IT Management!,5
7,7,I was disappointed because the name is mislead...,3
8,8,Super content. I'll definitely re-do the course,5
9,9,One of the excellent courses at Coursera for i...,5


In [150]:
dataset.tail()

,Id,Review,Label
107013,107013,Trendy topic with talks from expertises in the...,4
107014,107014,"Wonderful! Simple and clear language, good ins...",5
107015,107015,an interesting and fun course. thanks. dr quincy,5
107016,107016,"very broad perspective, up to date information...",4
107017,107017,An informative course on the social and financ...,4


In [151]:
def recode(label):
  if label<=2:
    new = 0
    label=new
  else:
    new = 1
    label=new
  return label

In [152]:
dataset.count()

Id        107018
Review    107018
Label     107018
dtype: int64

In [153]:
dataset['Label'].value_counts()

5    79173
4    18054
3     5071
1     2469
2     2251
Name: Label, dtype: int64

Data Splitted into Training and Test

In [154]:
training_set = pd.read_csv('https://raw.githubusercontent.com/PersieB/AshTutor/master/Training_set.csv')


In [134]:
training_set['Label'] = training_set['Label'].apply(recode)

In [155]:
training_set.head()

,Id,Review,Label
0,0,good and interesting,5
1,1,"This class is very helpful to me. Currently, I...",5
2,2,like!Prof and TAs are helpful and the discussi...,5
3,3,Easy to follow and includes a lot basic and im...,5
4,4,Really nice teacher!I could got the point eazl...,4


In [156]:
training_set.count()

Id        85614
Review    85614
Label     85614
dtype: int64

In [157]:
test = pd.read_csv('https://raw.githubusercontent.com/PersieB/AshTutor/master/Test_set.csv')
test.head()

,Id,Review,Label
0,85614,wonderful experience,5
1,85615,I took this course about 10 years off the coll...,5
2,85616,This series is really the course you need to t...,5
3,85617,Not as good as part (a). But still a great class!,4
4,85618,Incredible continuation of an already amazing ...,5


In [158]:
test['Label'] = test['Label'].apply(recode)

In [159]:
test.head()

,Id,Review,Label
0,85614,wonderful experience,1
1,85615,I took this course about 10 years off the coll...,1
2,85616,This series is really the course you need to t...,1
3,85617,Not as good as part (a). But still a great class!,1
4,85618,Incredible continuation of an already amazing ...,1


In [161]:
test.count()

Id        21404
Review    21404
Label     21404
dtype: int64

Remove punctuations

In [162]:
import string
def remove_punctuation(review):
  non_punctuated = [x for x in review if x not in string.punctuation]
  non_punctuated = ''.join(non_punctuated)
  return non_punctuated

Nltk stopwords

Remove stopwords

In [165]:
from nltk.corpus import stopwords
def remove_stopwords(reviews):
  stop_words = set(stopwords.words('english')) - set(['not', 'off'])
  return [word for word in reviews.split() if word.lower() not in stop_words]

Tokenize reviews

In [166]:
def tokenization(review):
  s = remove_punctuation(review)
  tokens = remove_stopwords(s)
  tokens = [i.lower() for i in tokens]
  return tokens

Naive Bayes Algorithm

function **train** which implements the naive Bayes algorithm. It takes in parameters D (the training document) and C (a list containing the classes) and returns the loglikelihood, logprior and V (a set of vocabulary)

In [167]:
def training(D, C):
  N_doc = len(D)  #number of documents (reviews) in D
  #N_c = {}  #number of documents from D in class c
  #intitialising dictionary to hold logpriors
  log_priors = {x:0 for x in C}

  #intitialising dictionary to hold loglikelihoods
  log_likelihood = {x:{} for x in C} 

  Vocabulary = [] #Vocabulary initialised empty
  big_doc_c = {k:{} for k in C} 
  D['Review'] = D['Review'].apply(lambda review: tokenization(review))

  for c in C:
    class_reviews = training_set[training_set['Label']==c]  # all reviews belonging to a class
    N_c = len(class_reviews)  # number of reviews belonging to a particular class
    log_priors[c] = math.log(N_c/N_doc) # log prior of c

    for i, j in class_reviews.iterrows():
      temp = j['Review']
      for word in temp:
        if word not in big_doc_c[c].keys():
          big_doc_c[c][word] = 0  # append(d) for d in D with class c
        big_doc_c[c][word] += 1   # increase word count
        Vocabulary.append(word)
  
  count = 0
  for c in C:
    count = sum(big_doc_c[c].values())
    Vocabulary = set(Vocabulary)  # only unique words in vocabulary
    for token in Vocabulary:
      count_w_c_in_V = 0
      if token in big_doc_c[c]:
        count_w_c_in_V = big_doc_c[c][token]
      log_likelihood[c][token] = math.log((count_w_c_in_V + 1)/(count + len(Vocabulary))) #loglikelihoods for each word in a partlicular class
  return log_priors, log_likelihood, Vocabulary

Training

In [168]:
C = np.unique(dataset['Label'].values)
logprior, loglikelihood, V = training(training_set, C)

function** predict** which implements the prediction or test function for the naive Bayes algorithm.

In [169]:
def predict(testdoc, logprior, loglikelihood, C, V):
  sum_c = {c:logprior[c] for c in C}
  for c in C:
    for word in tokenization(testdoc):
      if word in V:
        sum_c[c] += loglikelihood[c][word]

 #arg_max of sum_c  #key (class) with the largest value
  v = list(sum_c.values())
  k = list(sum_c.keys())
  chosen_class = k[v.index(max(v))] #return the key with the largest values
  if(chosen_class == 0):
    return 0
  else:
    return 1

Testing

In [170]:
predictions = []
for i in test['Review'].values:
  sentiment = predict(i, logprior, loglikelihood, C, V)
  predictions.append(sentiment)
  #print(i, sentiment)
test['Implemented Naive Bayes'] = predictions

In [171]:
test.tail(10)

,Id,Review,Label,Implemented Naive Bayes
21394,107008,The great course. I hope that prof. Brook we c...,1,1
21395,107009,Excellent course!Will use information to help ...,1,1
21396,107010,"For those interested in taking exams, these qu...",0,1
21397,107011,This course is awesome. It introduce the Zika ...,1,1
21398,107012,"Great, clear, concise, explanations on everyth...",1,1
21399,107013,Trendy topic with talks from expertises in the...,1,1
21400,107014,"Wonderful! Simple and clear language, good ins...",1,1
21401,107015,an interesting and fun course. thanks. dr quincy,1,1
21402,107016,"very broad perspective, up to date information...",1,1
21403,107017,An informative course on the social and financ...,1,1


In [172]:
sen = predict('very bad tutorial', logprior, loglikelihood, C, V)
print(sen)

1
